# Mengimport Spark SQL, library untuk machine learning dan membuat session

In [1]:
!pip3 install pyspark


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:07
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 1.1 MB/s eta 0:00:001.2 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824009 sha256=41f5bf33e97215f6dbcac5b5f64a9de6a6ee338469438c9152933a7e67741990
  Stored in directory: /home/riski/.cache/pip/wheels/89/d6/52/1178e354ba2207673484f0ccd7b2ded0ab6671ae5c1fc5b49a
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

# membuat session
appName = "Klasifikasi di Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


23/04/07 03:03:51 WARN Utils: Your hostname, riski-Aspire-E5-473G resolves to a loopback address: 127.0.1.1; using 192.168.1.16 instead (on interface wlx1c61b4150d77)
23/04/07 03:03:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/07 03:03:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#mengimport modul yang dibutuhkan
from pyspark.sql import SparkSession

# membuat session
appName = "Klasifikasi di Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

#membuat session
appName = "Klasifikasi di Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Memuat data dari file

In [8]:
#membuat skema data file kita
flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
])

#membaca file ke DataFrame dg skema yg telah dibuat sblmnya
csv = spark.read.csv('../dataset/customers.csv', schema=flightSchema, header=True)
csv.show(3)

23/04/07 03:07:46 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 11, schema size: 7
CSV file: file:///home/riski/Dokumen/Data_Mining/Data-Mining-and-Big-Data-Analytics-Book-master/dataset/customers.csv


+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|      null|       42|      0|          50000|            0|       0|       0|
|      null|       40|      1|          50000|            0|       0|       0|
|      null|       63|      0|          25000|            0|       2|       1|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 3 rows



# Menyiapkan data

In [9]:
data = csv.select(
    "DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", 
    "DepDelay", ((col("ArrDelay") > 15).cast("Int").alias("Late")))
data.show(3)

23/04/07 03:09:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CustomerName, Age, IncomeRange, Gender, TotalChildren, ChildrenAtHome
 Schema: DayofMonth, DayOfWeek, OriginAirportID, DestAirportID, DepDelay, ArrDelay
Expected: DayofMonth but found: CustomerName
CSV file: file:///home/riski/Dokumen/Data_Mining/Data-Mining-and-Big-Data-Analytics-Book-master/dataset/customers.csv
+----------+---------+---------------+-------------+--------+----+
|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|Late|
+----------+---------+---------------+-------------+--------+----+
|      null|       42|          50000|            0|       0|   0|
|      null|       40|          50000|            0|       0|   0|
|      null|       63|          25000|            0|       2|   0|
+----------+---------+---------------+-------------+--------+----+
only showing top 3 rows



# Membagi data training dan testing

In [10]:
#membagi data training dan data testing
dataTerpisahkan = data.randomSplit([0.7, 0.3])
train = dataTerpisahkan[0]
test = dataTerpisahkan[1]
train_rows = train.count()
test_rows = test.count()
print ("Jumlah baris data training:", 
       train_rows, " Jumlah baris data testing:", test_rows)

23/04/07 03:09:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CustomerName, Age, IncomeRange, Gender, TotalChildren, ChildrenAtHome
 Schema: DayofMonth, DayOfWeek, OriginAirportID, DestAirportID, DepDelay, ArrDelay
Expected: DayofMonth but found: CustomerName
CSV file: file:///home/riski/Dokumen/Data_Mining/Data-Mining-and-Big-Data-Analytics-Book-master/dataset/customers.csv


23/04/07 03:09:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CustomerName, Age, IncomeRange, Gender, TotalChildren, ChildrenAtHome
 Schema: DayofMonth, DayOfWeek, OriginAirportID, DestAirportID, DepDelay, ArrDelay
Expected: DayofMonth but found: CustomerName
CSV file: file:///home/riski/Dokumen/Data_Mining/Data-Mining-and-Big-Data-Analytics-Book-master/dataset/customers.csv


Jumlah baris data training: 12941  Jumlah baris data testing: 5567


# Menyiapkan data training

In [11]:
train.show(2)
assembler = VectorAssembler(inputCols = [
    "DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", 
    "DepDelay"], outputCol="features")
trainingDataFinal = assembler.transform(
    train).select(col("features"), col("Late").alias("label"))
trainingDataFinal.show(truncate=False, n=2)

23/04/07 03:10:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CustomerName, Age, IncomeRange, Gender, TotalChildren, ChildrenAtHome
 Schema: DayofMonth, DayOfWeek, OriginAirportID, DestAirportID, DepDelay, ArrDelay
Expected: DayofMonth but found: CustomerName
CSV file: file:///home/riski/Dokumen/Data_Mining/Data-Mining-and-Big-Data-Analytics-Book-master/dataset/customers.csv


+----------+---------+---------------+-------------+--------+----+
|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|Late|
+----------+---------+---------------+-------------+--------+----+
|      null|       36|          25000|            0|       0|   0|
|      null|       36|          25000|            0|       0|   0|
+----------+---------+---------------+-------------+--------+----+
only showing top 2 rows

23/04/07 03:10:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CustomerName, Age, IncomeRange, Gender, TotalChildren, ChildrenAtHome
 Schema: DayofMonth, DayOfWeek, OriginAirportID, DestAirportID, DepDelay, ArrDelay
Expected: DayofMonth but found: CustomerName
CSV file: file:///home/riski/Dokumen/Data_Mining/Data-Mining-and-Big-Data-Analytics-Book-master/dataset/customers.csv


23/04/07 03:10:10 ERROR Executor: Exception in task 0.0 in stage 9.0 (TID 7)
org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3389/0x0000000801e96120: (struct<DayofMonth_double_VectorAssembler_e29c048c2559:double,DayOfWeek_double_VectorAssembler_e29c048c2559:double,OriginAirportID_double_VectorAssembler_e29c048c2559:double,DestAirportID_double_VectorAssembler_e29c048c2559:double,DepDelay_double_VectorAssembler_e29c048c2559:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowI

Py4JJavaError: An error occurred while calling o118.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 7) (192.168.1.16 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3389/0x0000000801e96120: (struct<DayofMonth_double_VectorAssembler_e29c048c2559:double,DayOfWeek_double_VectorAssembler_e29c048c2559:double,OriginAirportID_double_VectorAssembler_e29c048c2559:double,DestAirportID_double_VectorAssembler_e29c048c2559:double,DepDelay_double_VectorAssembler_e29c048c2559:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spa

# Mentraining model klasifikasi

In [12]:
#mendefinisikan algoritma klasifikasi kita
algoritmaKlasifikasi = LogisticRegression(
    labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
#mentraining model algoritma klasifikasi kita
model = algoritmaKlasifikasi.fit(trainingDataFinal)
print ("Model selesai ditraining!")

23/04/07 03:10:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CustomerName, Age, IncomeRange, Gender, TotalChildren, ChildrenAtHome
 Schema: DayofMonth, DayOfWeek, OriginAirportID, DestAirportID, DepDelay, ArrDelay
Expected: DayofMonth but found: CustomerName
CSV file: file:///home/riski/Dokumen/Data_Mining/Data-Mining-and-Big-Data-Analytics-Book-master/dataset/customers.csv


23/04/07 03:10:21 ERROR Executor: Exception in task 0.0 in stage 10.0 (TID 8)
org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3389/0x0000000801e96120: (struct<DayofMonth_double_VectorAssembler_e29c048c2559:double,DayOfWeek_double_VectorAssembler_e29c048c2559:double,OriginAirportID_double_VectorAssembler_e29c048c2559:double,DestAirportID_double_VectorAssembler_e29c048c2559:double,DepDelay_double_VectorAssembler_e29c048c2559:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRow

Py4JJavaError: An error occurred while calling o122.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 8) (192.168.1.16 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3389/0x0000000801e96120: (struct<DayofMonth_double_VectorAssembler_e29c048c2559:double,DayOfWeek_double_VectorAssembler_e29c048c2559:double,OriginAirportID_double_VectorAssembler_e29c048c2559:double,DestAirportID_double_VectorAssembler_e29c048c2559:double,DepDelay_double_VectorAssembler_e29c048c2559:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sq

# Menyiapkan data testing

In [13]:
testingDataFinal = assembler.transform(
    test).select(col("features"), col("Late").alias("trueLabel"))
testingDataFinal.show(3)

23/04/07 03:10:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CustomerName, Age, IncomeRange, Gender, TotalChildren, ChildrenAtHome
 Schema: DayofMonth, DayOfWeek, OriginAirportID, DestAirportID, DepDelay, ArrDelay
Expected: DayofMonth but found: CustomerName
CSV file: file:///home/riski/Dokumen/Data_Mining/Data-Mining-and-Big-Data-Analytics-Book-master/dataset/customers.csv


23/04/07 03:10:35 ERROR Executor: Exception in task 0.0 in stage 11.0 (TID 9)
org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3389/0x0000000801e96120: (struct<DayofMonth_double_VectorAssembler_e29c048c2559:double,DayOfWeek_double_VectorAssembler_e29c048c2559:double,OriginAirportID_double_VectorAssembler_e29c048c2559:double,DestAirportID_double_VectorAssembler_e29c048c2559:double,DepDelay_double_VectorAssembler_e29c048c2559:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRow

Py4JJavaError: An error occurred while calling o191.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 9) (192.168.1.16 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3389/0x0000000801e96120: (struct<DayofMonth_double_VectorAssembler_e29c048c2559:double,DayOfWeek_double_VectorAssembler_e29c048c2559:double,OriginAirportID_double_VectorAssembler_e29c048c2559:double,DestAirportID_double_VectorAssembler_e29c048c2559:double,DepDelay_double_VectorAssembler_e29c048c2559:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.s

# Mengetes data dg model yang telah kita training

In [14]:
prediksiMentah = model.transform(testingDataFinal)
prediksiFinal = prediksiMentah.select(
    "features", "prediction", "probability", "trueLabel")
prediksiFinal.show(truncate=False, n=3)

NameError: name 'model' is not defined

# Mengevaluasi seberapa akurat model klasifikasi kita

In [9]:
prediksiBenar = prediksiFinal.filter(
    prediksiFinal['prediction'] == prediksiFinal['trueLabel']).count()
totalData = prediksiFinal.count()
print("prediksi benar: ", prediksiBenar, ", total data: ", totalData, 
      ", akurasi: ", prediksiBenar/totalData)

prediksi benar:  668630 , total data:  810877 , akurasi:  0.8245763537503222


# Mencoba menggunakan algoritma klasifikasi lain

In [10]:
from pyspark.ml.classification import RandomForestClassifier

algoritmaKlasifikasi = RandomForestClassifier(
    numTrees=3, maxDepth=5, seed=42, labelCol="label",featuresCol="features")
model = algoritmaKlasifikasi.fit(trainingDataFinal)
print ("Model selesai ditraining!")

Model selesai ditraining!


In [11]:
prediksiMentah = model.transform(testingDataFinal)
prediksiFinal = prediksiMentah.select(
    "features", "prediction", "probability", "trueLabel")
prediksiFinal.show(truncate=False, n=3)#menampilkan 3 hasil prediksi data
prediksiBenar = prediksiFinal.filter(
    prediksiFinal['prediction'] == prediksiFinal['trueLabel']).count()
totalData = prediksiFinal.count()
print("prediksi benar: ", prediksiBenar, ", total data: ", 
      totalData, ", akurasi: ", prediksiBenar/totalData)

+------------------------------+----------+----------------------------------------+---------+
|features                      |prediction|probability                             |trueLabel|
+------------------------------+----------+----------------------------------------+---------+
|[1.0,1.0,10140.0,10397.0,-2.0]|0.0       |[0.9499887351533095,0.05001126484669058]|0        |
|[1.0,1.0,10140.0,11259.0,0.0] |0.0       |[0.9499887351533095,0.05001126484669058]|0        |
|[1.0,1.0,10140.0,11259.0,21.0]|1.0       |[0.45300585244219765,0.5469941475578023]|1        |
+------------------------------+----------+----------------------------------------+---------+
only showing top 3 rows

prediksi benar:  751035 , total data:  810877 , akurasi:  0.9262008911339205
